In [27]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [28]:
import json
from pathlib import Path

import torch

baseline_path = Path("/bigdata/cifar10/logs/garipov/baselines/1647358615/gar_18_dNone_128_sgd_l0.1_g0.1_w0.0_sTrue")
baseline_model = torch.load(baseline_path / "cnn_best.pth")
with open(baseline_path/'results.json') as json_file:
    baseline_result = json.load(json_file)
baseline_result

{'best_epoch': 38,
 'best_train_acc': 0.9749333333333333,
 'best_valid_acc': 0.9078,
 'best_valid_loss': 0.003568341651931405,
 'test_acc': 0.891,
 'test_loss': 0.004007616302371025,
 'n_param': 557642,
 'model_name': 'gar_18_dNone_128_sgd_l0.1_g0.1_w0.0_sTrue'}

In [29]:
baseline_model

GaripovNet(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv1_bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2_bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3_bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv4): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4_bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv5): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv5_bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv6): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=128, out_features=1

In [21]:
from tddl.factorizations import number_layers, listify_numbered_layers, get_weights

numbered_layers = number_layers(baseline_model)
gar_layers = [2,4,6,8,10]
layers = listify_numbered_layers(numbered_layers, layer_nrs=gar_layers)

In [22]:
weights = get_weights(baseline_model, gar_layers)

In [23]:
from tddl.dbs import find_rank_given_error

find_rank_given_error(layers[0][2], desired_error = 0.5)

factorize_layer.cache_info() = CacheInfo(hits=0, misses=38, maxsize=4096, currsize=38)
count_parameters.cache_info() = CacheInfo(hits=0, misses=8, maxsize=128, currsize=8)


(7787, 0.203125, tensor(0.5003, device='cuda:0'))

In [24]:
from tddl.dbs import compress_layers_with_desired_error
from tddl.utils.model_stats import count_parameters

baseline_count = count_parameters(baseline_model)

compress_layers_with_desired_error(layers, desired_error=0.5, baseline_count=baseline_count)

factorize_layer.cache_info() = CacheInfo(hits=0, misses=43, maxsize=4096, currsize=43)
count_parameters.cache_info() = CacheInfo(hits=0, misses=10, maxsize=128, currsize=10)
factorize_layer.cache_info() = CacheInfo(hits=0, misses=47, maxsize=4096, currsize=47)
count_parameters.cache_info() = CacheInfo(hits=0, misses=11, maxsize=128, currsize=11)
factorize_layer.cache_info() = CacheInfo(hits=0, misses=50, maxsize=4096, currsize=50)
count_parameters.cache_info() = CacheInfo(hits=0, misses=12, maxsize=128, currsize=12)
factorize_layer.cache_info() = CacheInfo(hits=0, misses=58, maxsize=4096, currsize=58)
count_parameters.cache_info() = CacheInfo(hits=0, misses=13, maxsize=128, currsize=13)
find_rank_given_error.cache_info() = CacheInfo(hits=2, misses=11, maxsize=128, currsize=11)


(0.2735034305163528,
 [0.203125, 0.28125, 0.4375, 0.375, 0.02734375],
 tensor(0.5044, device='cuda:0'))

In [25]:
from tddl.dbs import find_error_given_c
from tddl.utils.model_stats import count_parameters

baseline_count = count_parameters(baseline_model)


In [26]:
# with cache
%time ranks, c, error = find_error_given_c(layers, desired_c = 0.5, baseline_count=baseline_count)

0.5
factorize_layer.cache_info() = CacheInfo(hits=6, misses=58, maxsize=4096, currsize=58)
count_parameters.cache_info() = CacheInfo(hits=2, misses=13, maxsize=128, currsize=13)
factorize_layer.cache_info() = CacheInfo(hits=11, misses=58, maxsize=4096, currsize=58)
count_parameters.cache_info() = CacheInfo(hits=3, misses=13, maxsize=128, currsize=13)
factorize_layer.cache_info() = CacheInfo(hits=15, misses=58, maxsize=4096, currsize=58)
count_parameters.cache_info() = CacheInfo(hits=4, misses=13, maxsize=128, currsize=13)
factorize_layer.cache_info() = CacheInfo(hits=18, misses=58, maxsize=4096, currsize=58)
count_parameters.cache_info() = CacheInfo(hits=5, misses=13, maxsize=128, currsize=13)
factorize_layer.cache_info() = CacheInfo(hits=26, misses=58, maxsize=4096, currsize=58)
count_parameters.cache_info() = CacheInfo(hits=6, misses=13, maxsize=128, currsize=13)
find_rank_given_error.cache_info() = CacheInfo(hits=2, misses=16, maxsize=128, currsize=16)
0.2735034305163528
0.226496569

In [19]:
# no cache
%time ranks, c, error = find_error_given_c(layers, desired_c = 0.5, baseline_count=baseline_count)

0.5
0.2735034305163528
0.22649656948364721
True
----------
0.25
0.6774041410080303
-0.17740414100803026
True
----------
0.375
0.4269280290939348
0.07307197090606521
True
----------
0.3125
0.5415122964195667
-0.04151229641956666
True
----------
0.34375
0.48262146681921375
0.01737853318078625
True
----------
0.328125
0.5128182597437065
-0.01281825974370654
True
----------
0.3359375
0.5011727954494102
-0.00117279544941018
False
CPU times: user 4min 38s, sys: 6min 42s, total: 11min 21s
Wall time: 43.1 s


In [10]:
from tddl.dbs import find_error_given_c
from tddl.utils.model_stats import count_parameters

baseline_count = count_parameters(baseline_model)
ranks, c, error = find_error_given_c(layers, desired_c = 0.5, baseline_count=baseline_count)

0.5


TypeError: unhashable type: 'list'

In [43]:
import numpy as np


rank = 0.5
errors = np.array([])
for layer in layers:
    with torch.no_grad():
        fact_layer, error = factorize_layer(layer[2], 'tucker', rank, return_error=True)
    errors = np.append(errors, float(error.cpu().numpy()))

In [46]:
error = errors.mean()
max_error = errors.max()
min_error = errors.min()

In [47]:
find_error_given_c(layers, desired_c = 0.5, error=error, max_error=max_error, min_error=min_error)

0.35240626633167266
-0.41636698440207964
0.9163669844020796
True
----------
0.4085878893733025
-0.1684683708838821
0.6684683708838821
True
----------
0.43667870089411737
-0.07165294627383023
0.5716529462738302
True
----------
0.4507241066545248
0.0006878249566724115
0.4993121750433276
True
----------
0.4577468095347285
0.035934792027729645
0.46406520797227035
True
----------
0.4612581609748304
0.043224653379549394
0.4567753466204506
True
----------
0.46301383669488133
0.06800259965337951
0.4319974003466205
True
----------
0.46389167455490676
0.06800259965337951
0.4319974003466205
True
----------
0.4643305934849195
0.06800259965337951
0.4319974003466205
True
----------
0.4645500529499259
0.06800259965337951
0.4319974003466205
True
----------
0.4646597826824291
0.06800259965337951
0.4319974003466205
True
----------
0.46471464754868064
0.06800259965337951
0.4319974003466205
True
----------
0.46474207998180644
0.06800259965337951
0.4319974003466205
True
----------
0.4647557961983694
0.0680

KeyboardInterrupt: 